## 카카오 지오코딩 완성

In [2]:
import pandas as pd
import re
import requests, json

# 위경도 찾을 파일 불러오기
file_path = './data/2.충전소/충전소_부산_제한있음.csv'
busan_charge = pd.read_csv(file_path)

# 주소 전처리
pattern = r'\([^)]*\)'
text = []

for i in range(0, len(busan_charge['주소']),1):
    text.append(re.sub(pattern=pattern, repl='', string= busan_charge['주소'][i]))

text
busan_charge['주소'] = text

# 카카오API를 사용하여 주소 -> 좌표 변환
def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 57da1f1bcfa0f4b06f0350e852287856"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

# 파일에서 주소를 위경도로 변환해서 crd에 넣기
crd = []

for i in range(0, len(busan_charge['주소']), 1):
    try:
        crd.append(get_location(busan_charge['주소'][i]))
    except:
        crd.append({'lat':0,'lng':0})

# crd를 데이터프레임화 하여 원래 파일에 붙이기
busan = pd.DataFrame(crd)
busan_charge = pd.concat([busan_charge,busan], axis=1)
busan_charge

,Unnamed: 0,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2,lat,lng
0,0,파킹클라우드,부산 동트레아파트,1,AC완속,부산광역시,수영구,부산광역시 수영구 무학로9번길 15-1,거주자외 출입제한,무학로9번길 15-1(동트레 아파트),35.1625296741165,129.114706008742
1,1,파킹클라우드,부산 동트레아파트,2,AC완속,부산광역시,수영구,부산광역시 수영구 무학로9번길 15-1,거주자외 출입제한,무학로9번길 15-1(동트레 아파트),35.1625296741165,129.114706008742
2,2,제주전기자동차서비스,경보이리스오션더스타2차B1,1,DC차데모+AC3상+DC콤보,부산광역시,중구,부산광역시 중구 대교로 133경보이리스오션더스타2차 B1,시설 상황에 따라 이용이 제한될 수 있음,교로 133경보이리스오션더스타2차 B1,35.1019447696819,129.037408230096
3,3,제주전기자동차서비스,금정산SKTVIEW 아파트 101동 A주차장 62번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64101동 A주차장 62번 기둥,거주자외 출입제한,식물원로 64101동 A주차장 62번 기둥(713 금정산에스케이뷰아파트),0,0
4,4,제주전기자동차서비스,금정산SKTVIEW 아파트 102동 A주차장 29번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64102동 A주차장 29번 기둥,거주자외 출입제한,식물원로 64102동 A주차장 29번 기둥(713 금정산에스케이뷰아파트),0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4791,4791,한국전자금융,롯데월드 어드벤처 부산,4,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4792,4792,한국전자금융,롯데월드 어드벤처 부산,5,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4793,4793,한국전자금융,롯데월드 어드벤처 부산,6,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4794,4794,한국전자금융,롯데월드 어드벤처 부산,7,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811


In [3]:
busan_charge_else = busan_charge.drop(columns='Unnamed: 0')
busan_charge_else

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2,lat,lng
0,파킹클라우드,부산 동트레아파트,1,AC완속,부산광역시,수영구,부산광역시 수영구 무학로9번길 15-1,거주자외 출입제한,무학로9번길 15-1(동트레 아파트),35.1625296741165,129.114706008742
1,파킹클라우드,부산 동트레아파트,2,AC완속,부산광역시,수영구,부산광역시 수영구 무학로9번길 15-1,거주자외 출입제한,무학로9번길 15-1(동트레 아파트),35.1625296741165,129.114706008742
2,제주전기자동차서비스,경보이리스오션더스타2차B1,1,DC차데모+AC3상+DC콤보,부산광역시,중구,부산광역시 중구 대교로 133경보이리스오션더스타2차 B1,시설 상황에 따라 이용이 제한될 수 있음,교로 133경보이리스오션더스타2차 B1,35.1019447696819,129.037408230096
3,제주전기자동차서비스,금정산SKTVIEW 아파트 101동 A주차장 62번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64101동 A주차장 62번 기둥,거주자외 출입제한,식물원로 64101동 A주차장 62번 기둥(713 금정산에스케이뷰아파트),0,0
4,제주전기자동차서비스,금정산SKTVIEW 아파트 102동 A주차장 29번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64102동 A주차장 29번 기둥,거주자외 출입제한,식물원로 64102동 A주차장 29번 기둥(713 금정산에스케이뷰아파트),0,0
...,...,...,...,...,...,...,...,...,...,...,...
4791,한국전자금융,롯데월드 어드벤처 부산,4,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4792,한국전자금융,롯데월드 어드벤처 부산,5,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4793,한국전자금융,롯데월드 어드벤처 부산,6,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811
4794,한국전자금융,롯데월드 어드벤처 부산,7,AC완속,부산광역시,기장군,부산광역시 기장군 기장읍 동부산관광로 42롯데월드 어드벤처 부산,시설 상황에 따라 이용이 제한될 수 있음,기장읍 동부산관광로 42롯데월드 어드벤처 부산(시랑리 산46),35.1960618136802,129.213206917811


In [41]:
clear = pd.read_excel('./data/2.충전소/부산_결측치.xlsx')
clear

,Unnamed: 0,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2,lat,lng
0,3,제주전기자동차서비스,금정산SKTVIEW 아파트 101동 A주차장 62번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64101동 A주차장 62번 기둥,거주자외 출입제한,식물원로 64101동 A주차장 62번 기둥(713 금정산에스케이뷰아파트),35.227397,129.079321
1,4,제주전기자동차서비스,금정산SKTVIEW 아파트 102동 A주차장 29번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64102동 A주차장 29번 기둥,거주자외 출입제한,식물원로 64102동 A주차장 29번 기둥(713 금정산에스케이뷰아파트),35.227397,129.079321
2,5,제주전기자동차서비스,금정산SKTVIEW 아파트 103동 A주차장 04번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64103동 A주차장 04번 기둥,거주자외 출입제한,식물원로 64103동 A주차장 04번 기둥(713 금정산에스케이뷰아파트),35.227397,129.079321
3,6,제주전기자동차서비스,금정산SKTVIEW 아파트 104동 D주차장 58번 기둥,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64104동 D주차장 58번 기둥,거주자외 출입제한,식물원로 64104동 D주차장 58번 기둥(713 금정산에스케이뷰아파트),35.227397,129.079321
4,7,제주전기자동차서비스,금정산SKTVIEW 아파트 106동 D주차장 20번 기둥 A,1,AC완속,부산광역시,금정구,부산광역시 금정구 식물원로 64106동 D주차장 20번 기둥 A,거주자외 출입제한,식물원로 64106동 D주차장 20번 기둥 A(713 금정산에스케이뷰아파트),35.227397,129.079321
...,...,...,...,...,...,...,...,...,...,...,...,...
236,4642,블루네트웍스,동래 래미안아이파크 2단지,1,DC콤보,부산광역시,동래구,"부산광역시 동래구 금정마을로 1502단지 지하2층,지하3층 주차장,지하3층 주차장)",거주자 및 방문자 사용가능,"금정마을로 1502단지 지하2층(201동),지하3층(209동) 주차장(래미안 아이파...",35.211280,129.067547
237,4643,블루네트웍스,동래 래미안아이파크 2단지,2,DC콤보,부산광역시,동래구,"부산광역시 동래구 금정마을로 1502단지 지하2층,지하3층 주차장,지하3층 주차장)",거주자 및 방문자 사용가능,"금정마을로 1502단지 지하2층(201동),지하3층(209동) 주차장(래미안 아이파...",35.211280,129.067547
238,4644,블루네트웍스,동래 래미안아이파크 3단지,1,DC콤보,부산광역시,동래구,"부산광역시 동래구 충렬대로107번길 653단지 지하2층,지하3층 주차장,지하3층 주차장)",거주자 및 방문자 사용가능,"충렬대로107번길 653단지 지하2층(302동),지하3층(308동) 주차장(래미안 ...",35.210801,129.071334
239,4645,블루네트웍스,동래 래미안아이파크 3단지,2,DC콤보,부산광역시,동래구,"부산광역시 동래구 충렬대로107번길 653단지 지하2층,지하3층 주차장,지하3층 주차장)",거주자 및 방문자 사용가능,"충렬대로107번길 653단지 지하2층(302동),지하3층(308동) 주차장(래미안 ...",35.210801,129.071334


In [36]:
no_index = busan_charge_else[busan_charge_else['lat'] == 0].index

In [37]:
no_list = [3,    4,    5,    6,    7,    8,    9,   10,   11,   12,   13,
         14,   15,   16,   17,   62,   63,   64,   65,   66,   67,   68,
        203,  204,  205,  206,  226,  227,  228,  229,  230,  231,  236,
        237,  238,  241,  339,  340,  383,  394,  412,  413,  634,  635,
        636,  637,  638,  639,  660,  661,  662,  663,  664,  665,  666,
        667,  668,  669,  670,  671,  672,  760,  761,  762,  763,  764,
        809,  810,  811,  848,  849,  850,  851,  852, 1131, 1179, 1180,
       1181, 1182, 1183, 1184, 1201, 1202, 1203, 1214, 1215, 1216, 1217,
       1218, 1219, 1249, 1250, 1251, 1252, 1253, 1280, 1281, 1282, 1283,
       1284, 1285, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1348,
       1349, 1350, 1351, 1384, 1385, 1386, 1387, 1388, 1389, 1400, 1401,
       1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1443, 1444, 1445,
       1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456,
       1457, 1458, 1467, 1468, 1521, 1522, 1523, 1524, 1525, 1526, 1527,
       1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538,
       1539, 1540, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628,
       1629, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667,
       1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1732,
       1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743,
       1744, 1745, 1746, 1747, 1958, 1959, 2241, 2242, 2317, 2318, 2519,
       2520, 2521, 2522, 2523, 2524, 3477, 3478, 3481, 3482, 3483, 3484,
       3487, 3488, 3489, 4220, 4221, 4642, 4643, 4644, 4645, 4646]

In [43]:
busan_charge_else.iloc[no_list,-2] = clear['lat']
busan_charge_else.iloc[no_list,-1] = clear['lng']

In [46]:
busan_charge_else.to_csv('./data/2.충전소/부산_제한있음2.csv')
